In [1]:
import numpy as np
from numpy import load
from sklearn.metrics import (roc_curve, auc, accuracy_score, f1_score, precision_score,
                             recall_score, classification_report, confusion_matrix)
from skimage.feature import hog
import seaborn as sn
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from dataio import *
from preprocess import *
from segment import *
from colorfeatures import *
from classify import *
from feature_extraction import *

In [8]:
from numpy import load
cnn_nv = load('/features/aug_train_cnn_features_nv.npy', allow_pickle=True)

In [6]:
cnn_les = load('/features/aug_train_cnn_features_les.npy', allow_pickle=True)

In [9]:
cnn_nv.shape

(22880, 4608)

In [ ]:
import numpy as np
from numpy import load
cnn_nv = load('/features/full_VGG16_features_nv.npy', allow_pickle=True)
cnn_les = load('/features/full_VGG16_features_les.npy', allow_pickle=True)
hc_nv = load('/features/full_train_hand_crafted_features_nv.npy', allow_pickle=True)
hc_les = load('/features/full_train_hand_crafted_features_les.npy', allow_pickle=True)
meta_nv = load('/features/full_metafeatures_hand_crafted_features_nv.npy', allow_pickle=True)
meta_les = load('/features/full_metafeatures_hand_crafted_features_les.npy', allow_pickle=True)

In [ ]:
nv = np.concatenate((cnn_nv,hc_nv,meta_nv),axis=1)
les = np.concatenate((cnn_les, hc_les,meta_les),axis=1)

In [ ]:
X, Y               = concat_and_shuffled(class0=nv, class1=les, shuffled=False)
X                  = (X-np.mean(X,axis=0))/np.std(X,axis=0)

In [ ]:
np.nan_to_num(X, copy=False, nan=0.0, posinf=None, neginf=None)
np.isfinite(X).all()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 X, Y, test_size=0.3, random_state=42)

In [ ]:
class_weight = {0: 1, 1: 99}

In [ ]:
RFclassifier = RandomForestClassifier(n_estimators=2000, max_depth=20, max_features='auto', class_weight=class_weight,
                                      min_samples_split=10, min_samples_leaf=1, bootstrap=True, random_state=40)
RFclassifier.fit(X_train, y_train)

# ● Results (AUC, F1, Accuracy, Precision, Recall, Confusion Matrix)

In [ ]:
ROC(X_train,Y_train,X_test,Y_test,SVMclassifier, RFclassifier)

In [ ]:
RFaccuracy  = accuracy_score(y_test, RFclassifier.predict(X_test))
RFprecision = precision_score(y_test, RFclassifier.predict(X_test),  average="macro")
RFrecall    = recall_score(y_test, RFclassifier.predict(X_test),  average="macro")
RFF1        = f1_score(y_test, RFclassifier.predict(X_test),  average="macro")
print("---------------------------------")
print("Random Forest")
print("Accuracy = %0.4f"%RFaccuracy),  print("Precision = %0.4f"%RFprecision)
print("Recall = %0.4f"%RFrecall),
print("F1 Score = %0.4f"%RFF1)

In [ ]:
TN2, FP2, FN2, TP2 = confusion_matrix(y_test, RFclassifier.predict(X_test)).ravel()
CM2 = confusion_matrix(y_test, RFclassifier.predict(X_test))
CM2 = pd.DataFrame(CM2, index = ["LES (Actual)","NV (Actual)"],
                   columns = ["LES (Predicted)","NV (Predicted)"])

%matplotlib qt
plt.figure(figsize=[12,4.5])
plt.subplot(121)

plt.subplot(122)
sn.heatmap(CM2, annot=True, fmt='g')

print("Random Forest")
print("TP:{TP} | TN:{TN} | FP:{FP} | FN:{FN}".format(TP=TP2,TN=TN2,FP=FP2,FN=FN2))
print("Sensitivity: {:.3f}".format(TP2/(TP2+FN2)))
print("Specificity: {:.3f}".format(TN2/(TN2+FP2)))